In [12]:
import QuantLib as ql
from QuantLib import *
from collections import namedtuple
import math
import matplotlib.pyplot as plt
import numpy as np
% matplotlib inline

In [13]:
timestep = 360
length = 30 # in years
forward_rate = 0.05
day_count = ql.Thirty360()
todays_date = ql.Date(12, 1, 2018)

In [17]:
ql.Settings.instance().evaluationDate = todays_date

spot_curve = ql.FlatForward(todays_date, ql.QuoteHandle(ql.SimpleQuote(forward_rate)), day_count)
spot_curve_handle = ql.YieldTermStructureHandle(spot_curve)

In [22]:
today = todays_date
calendar = TARGET()
settlement = ql.Date(12, ql.January, 2018);
ql.Settings.instance().evaluationDate = today;
term_structure = ql.YieldTermStructureHandle(ql.FlatForward(settlement,0.04875825,ql.Actual365Fixed()))
index = ql.Euribor1Y(term_structure)

In [34]:
cash = {     (1,Weeks): 0.0023,
             (1,Months): 0.0023,
             (3,Months): 0.0023,
             (6,Months): 0.0023}
futures = { Date(21,3,2018): 99.765,
            Date(20,6,2018): 99.75,
            Date(19,9,2018): 99.73,
            Date(19,12,2018): 99.69,
            Date(20,3,2019): 99.605,
            Date(19,6,2019): 99.47,
            Date(18,9,2019): 99.3,
            Date(18,12,2019): 99.085 }
swaps = { (3,Years): 0.0079,
          (4,Years): 0.012,
          (5,Years): 0.0157,
          (6,Years): 0.01865,
          (7,Years): 0.0216,
          (8,Years): 0.0235,
          (9,Years): 0.0254,
          (10,Years): 0.0273,
          (15,Years): 0.0297,
          (20,Years): 0.0316,
          (25,Years): 0.0335,
          (30,Years): 0.0354}

for n,unit in cash.keys():
    cash[(n,unit)] = SimpleQuote(cash[(n,unit)])
for d in futures.keys():
    futures[d] = SimpleQuote(futures[d])
for n,unit in swaps.keys():
    swaps[(n,unit)] = SimpleQuote(swaps[(n,unit)])
    

dayCounter = Actual360()
settlementDays = 2
cashHelpers = [ DepositRateHelper(QuoteHandle(cash[(n,unit)]),
                                     Period(n,unit), settlementDays,
                                     calendar, ModifiedFollowing,
                                     False, dayCounter)
                   for n, unit in [(1,Weeks),(1,Months),(3,Months),
                                   (6,Months)] ]

dayCounter = Actual360()
months = 3
futuresHelpers = [ FuturesRateHelper(QuoteHandle(futures[d]),
                                     d, months,
                                     calendar, ModifiedFollowing,
                                     True, dayCounter,
                                     QuoteHandle(SimpleQuote(0.0)))
                   for d in futures.keys() ]

settlementDays = 2
fixedLegFrequency = Semiannual
fixedLegTenor = Period(6,Months)
fixedLegAdjustment = Unadjusted
fixedLegDayCounter = Thirty360()
floatingLegFrequency = Quarterly
floatingLegTenor = Period(3,Months)
floatingLegAdjustment = ModifiedFollowing
swapHelpers = [ SwapRateHelper(QuoteHandle(swaps[(n,unit)]),
                               Period(n,unit), calendar,
                               fixedLegFrequency, fixedLegAdjustment,
                               fixedLegDayCounter, Euribor3M())
                for n, unit in swaps.keys() ]


discountTermStructure = RelinkableYieldTermStructureHandle()
forecastTermStructure = RelinkableYieldTermStructureHandle()

helpers = cashHelpers[:2] + futuresHelpers + swapHelpers[1:]
liborCurve = PiecewiseFlatForward(settlement, helpers,
                                            Actual360())
print(libor.dates())
for c in libor.dates():
#    print(libor.discount(c))
    print(libor.rate(c0))


NameError: name 'libor' is not defined

In [33]:
CalibrationData = namedtuple("CalibrationData", 
                             "start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),
        CalibrationData(2, 4, 0.1108),
        CalibrationData(3, 3, 0.1070),
        CalibrationData(4, 2, 0.1021),
        CalibrationData(5, 1, 0.1000 )]

In [ ]:
model = ql.BlackKarasinski(term_structure);
engine = ql.TreeSwaptionEngine(model, 100)
swaptions = create_swaption_helpers(data, index, term_structure, engine)

optimization_method = ql.LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = ql.EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)

a, sigma =  model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

In [ ]:
# <!-- collapse=True -->
def create_swaption_helpers(data, index, term_structure, engine):
    swaptions = []
    fixed_leg_tenor = ql.Period(1, ql.Years)
    fixed_leg_daycounter = ql.Actual360()
    floating_leg_daycounter = ql.Actual360()
    for d in data:
        vol_handle = ql.QuoteHandle(ql.SimpleQuote(d.volatility))
        helper = ql.SwaptionHelper(ql.Period(d.start, ql.Years),
                                   ql.Period(d.length, ql.Years),
                                   vol_handle,
                                   index,
                                   fixed_leg_tenor,
                                   fixed_leg_daycounter,
                                   floating_leg_daycounter,
                                   term_structure
                                   )
        helper.setPricingEngine(engine)
        swaptions.append(helper)
    return swaptions    

def calibration_report(swaptions, data):
    print("-"*82)
    print("%15s %15s %15s %15s %15s" % \
    ("Model Price", "Market Price", "Implied Vol", "Market Vol", "Rel Error"))
    print("-"*82)
    cum_err = 0.0
    for i, s in enumerate(swaptions):
        model_price = s.modelValue()
        market_vol = data[i].volatility
        black_price = s.blackPrice(market_vol)
        rel_error = model_price/black_price - 1.0
        implied_vol = s.impliedVolatility(model_price,
                                          1e-5, 50, 0.0, 0.50)
        rel_error2 = implied_vol/market_vol-1.0
        cum_err += rel_error2*rel_error2
        
        print("%15.5f %15.5f %15.5f %15.5f %15.5f" % \
        (model_price, black_price, implied_vol, market_vol, rel_error))
    print("-"*82)
    print("Cumulative Error : %15.5f" % math.sqrt(cum_err))

In [ ]:
calibration_report(swaptions, data)

In [ ]:
bk_process = ql.BlackKarasinski(spot_curve_handle, a, sigma)
rng = ql.GaussianRandomSequenceGenerator(ql.UniformRandomSequenceGenerator(timestep, ql.UniformRandomGenerator()))
seq = ql.GaussianPathGenerator(bk_process, length, timestep, rng, False)